### Obtaining Data - Player Info and Match ID

This will be a short program to extract TFT player information from the Riot API.
The goal is to use the request URL of the Riot API to obtain player data which we will store in a JSON format.
The follow up is to them use this info to eventually obtain match information.



In [ ]:
## The Player Level will be of the format [tier, div] where
## tier = "CHALLENGER", "GRANDMASTER", "MASTER", "DIAMOND", "PLATNIUM", "GOLD", "SILVER", "BRONZE", "IRON"
## div = "I","II","III","IV" which will be ignored if str = "Challenger", "Grandmaster", "Master"


PLAYER_RANK = ["MASTER","III"]
PAGE_NUMBER = 6    # For any tier below masters the data is stored in pages of 200
API_KEY = ""

def Player_Division(PLAYER_RANK = [None ,"I"]):
    if PLAYER_RANK[0] == "CHALLENGER":
        return "https://na1.api.riotgames.com/tft/league/v1/challenger"
    elif PLAYER_RANK[0] == "GRANDMASTER":
        return "https://na1.api.riotgames.com/tft/league/v1/grandmaster"
    elif PLAYER_RANK[0] == "MASTER":
        return "https://na1.api.riotgames.com/tft/league/v1/master"
    else:
        return "https://na1.api.riotgames.com/tft/league/v1/entries/" + PLAYER_RANK[0] + "/" + PLAYER_RANK[1]

## The general format will be below. If we want to look at lower division then there are multiple pages of information
## In that case we can append "&page=n" where n is some int

if PLAYER_RANK[0] == "CHALLENGER" or PLAYER_RANK[0] == "GRANDMASTER" or PLAYER_RANK[0] == "MASTER":
    REQUEST_URL = Player_Division(PLAYER_RANK) + "?api_key=" + API_KEY
else:
    REQUEST_URL = Player_Division(PLAYER_RANK) + "?page=" + str(PAGE_NUMBER) + "&api_key=" + API_KEY


print(REQUEST_URL)

In [2]:
## Now with the link we will produce the JSON files with the data on the players
import json
import numpy as np
import pandas as pd
import urllib.request
import time
import csv

url = urllib.request.urlopen(REQUEST_URL)    
data = json.loads(url.read().decode())
print(data)
    

jsonString = json.dumps(data)
jsonFile = open("player.json", "w")
jsonFile.write(jsonString)
jsonFile.close()


NameError: name 'REQUEST_URL' is not defined

In [ ]:
## Storing the JSON file in a dataframe for easy access and management

df = pd.read_json("player.json")
df

In [ ]:
## There will need to be two methodologies to obtain summonerId
## Split between MASTERS and above vs below MASTERS level players
## For CHELLENGER, GRANDMASTER and MASTER we will require some more work to extract the info out
ID = []
if PLAYER_RANK[0] == "CHALLENGER" or PLAYER_RANK[0] == "GRANDMASTER" or PLAYER_RANK[0] == "MASTER":
    for i in range(len(df)):
         ID.append(df["entries"][i]["summonerId"])
else:
    for i in range(len(df)):
         ID.append(df["summonerId"][i])
            
ID


In [ ]:
## From here we obtain the puuid from the summonerId which will then allow us to get match history
## The number of request is capped at 20 per second or 100 per minute
## We will halt the for loop every so often so that the program can run all the way through

PUUID = []
for i in range(len(ID)):
    try:
        url = urllib.request.urlopen("https://na1.api.riotgames.com/tft/summoner/v1/summoners/" + ID[i] + "?api_key=" + API_KEY)    
        data = json.loads(url.read().decode())
        PUUID.append(data["puuid"])
    except:
        print("Error at " + str(i))
    if i % 10 == 0:
        time.sleep(10)
        print(i)
PUUID

In [ ]:
len(PUUID)

In [ ]:
## Saves the PUUID into csv files

with open('puuid.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(PUUID)

In [ ]:
## Reads the csv file and puts data here

with open('puuid.csv', newline='') as f:
    reader = csv.reader(f)
    puuid = list(reader)[0]

print(puuid)

In [ ]:
## We will find matche id via the PUUID saved
MatchID = set({})
NUM_OF_MATCHES = 20

for i in range(len(puuid)):
    try:
        PUUID_MATCH_URL = "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + puuid[i] + "/ids?count=" + str(NUM_OF_MATCHES) +"&api_key=" + API_KEY
        url = urllib.request.urlopen(PUUID_MATCH_URL)    
        data = json.loads(url.read().decode())
        MatchID.update(data)
    except:
        print("Error at " + str(i))
    if i % 10 == 0:
        time.sleep(10)
        print(i)


In [ ]:
## Input filename of which to save this batch of match id from the puuid
filename = '' + '.csv'
with open(filename, 'w') as f: 
    write = csv.writer(f) 
    write.writerow(MatchID)

### Obtaining Data - Match Information

From the different matchid .csv files we combine into one match id file

In [3]:
## Will store the match ids as a set to eliminate overlap
## File name for looping through and adding to match id
## Will result in a list of unique match ids
matchid = set({})
filename = ['master_matchid.csv',
           'challenger_matchid.csv',
           'grandmaster_matchid.csv']

for x in filename:
    with open(x, newline='') as f:
        reader = csv.reader(f)
        temp = list(reader)[0]
    matchid = matchid.union(set(temp))
    
matchid = list(matchid)

In [5]:
## Writing the Match IDs into a file which we can access
## The storing of these data in cvs is to allow for easy management with possibility to combine them
with open('matchid.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(matchid)

### Extracting match info via Riot API using match ID

This will be the longest process in the data extracting portion since the amount of data is magnitude greater in size than the other data extracted. Also there is a limit to the amount of queries that can be made for non-approved Riot members.

In [7]:
with open('matchid.csv', newline='') as f:
    reader = csv.reader(f)
    matchid = list(reader)[0]

print(len(matchid))

99640


In [ ]:
## The raw data of the matches from the Match ID will be requested and places into a list

match_info = []

for i in range(len(matchid)):
    try:
        MATCH_URL = "https://americas.api.riotgames.com/tft/match/v1/matches/" + matchid[i] + "?api_key=" + API_KEY
        url = urllib.request.urlopen(MATCH_URL)    
        data = json.loads(url.read().decode())
        match_info.append(data)
    except:
        print("Error at " + str(i))
    if i % 10 == 0:
        time.sleep(10)
        print(i)

print(len(match_info))

In [ ]:
## Export the data set as a JSON File by date

from datetime import datetime
today = datetime.today()

jsonString = json.dumps(match_info)
##jsonFile = open(today.strftime("%Y%m%d") + ".json", "w")
jsonFile = open("data.json", "w")
jsonFile.write(jsonString)
jsonFile.close()